# Azure OpenAI Assistants - Function Calling
Travel Agent Assistant

## Load Azure Configuration

In [6]:
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

# Load environment variables from .env file

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Functions

In [2]:
tools_function = [
    {
        "type": "function",
        "function": {
            "name": "get_countries",
            "description": "Retrieves the countries available."
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_travel_budget",
            "description": "Answers questions if a person has the money to visit a specific country. You will use this function when money is involved in the question",
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Retrieves the weather for a country.",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country (e.g., 'USA', 'France')."
                    }
                },
                "required": ["country"]
            }
        }
    },

]

# define functions
def get_countries():
    return """
        United States
        Australia
        France
    """

def get_travel_budget():
    return """
        If country is not defined:
            Look at all the budgets per country
        United States Budget:
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        Australia Budget:
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        France Budget:
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
def get_weather(country):
    if country == "United States":
        return """
            The United States has a diverse climate. In general:
            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
            - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
        """
    elif country == "Australia":
        return """
            Australia experiences varied climates:
            - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
            - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
        """
    elif country == "France":
        return """
            France has several climate zones:
            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
            - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
        """
    else:
        return "Country not found"

## Step 1-2:
1. Create an Assistant
2. Create a Thread

In [7]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

# Step 1: Create an assistant
#	Assistant is a custom AI that uses Azure OpenAI models in conjunction with tools.
# For other parameters: https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-reference?tabs=python#create-an-assistant
assistant = client.beta.assistants.create(
  name="Travel Assistant",
  instructions="""You are an assistant that answers questions about travel.""",
  model=azure_openai_deployment, 
  tools=tools_function
)

# Step 2: Create a thread
# Thread is a conversation session between an Assistant and a user. 
# Threads store Messages and automatically handle truncation to fit content into a model’s context.
# https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-reference-threads?tabs=python
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_1c9y0nqudbbjx6akALPWAVc2', created_at=1750531861, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## Step 3-6: Helper Function
3. Add a message to the thread
4. Run the Assistant
5. Check the Run Status
6. Display the Assistant's Response

In [8]:
import time
import json

def run_assistant(user_question):
  # Step 3: Add a message/user question to the thread
  # Message is a message created by an Assistant or a user. 
  # Messages can include text, images, and other files. 
  # Messages are stored as a list on the Thread.
  messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_question
  )

  # Step 4: Run the Assistant
  # Run is an activation of an Assistant to begin running based on the contents of the Thread. 
  # The Assistant uses its configuration and the Thread’s Messages to perform tasks by calling models and tools. 
  # As part of a Run, the Assistant appends Messages to the Thread.
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
  )

  # Step 5: Check the Run Status - Looping until the run completes or fails
  # When you initiate a Run with a user Message that triggers the function, the Run will enter a "in_progress" status. 
  # After it processes, the run will enter a "requires_action" state that you can verify by retrieving the Run.
  while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )

    #display run status
    print(run.status)

    if run.status == 'completed':
      messages = client.beta.threads.messages.list(thread_id=thread.id)

      # Step 6: Display the Assistant's Response
      content_block = messages.data[0].content[0]
      value = content_block.text.value
      print(value)
    
    elif run.status == 'requires_action':
      tool_outputs = []
      for tool_call in run.required_action.submit_tool_outputs.tool_calls:
        function_name = tool_call.function.name
        print("Function Name: ", function_name)
        
        function_args = json.loads(tool_call.function.arguments)
        print("Function Arguments: ", function_args)    
        
        if function_name == "get_countries":
          function_response = get_countries()
        elif function_name == "get_travel_budget":
          function_response = get_travel_budget()
        elif function_name == "get_weather":
          function_response = get_weather(country=function_args.get("country"))    
        else:
          function_response = "Unknown function"
        print("Function Response: ", function_response)

        ### You can then complete the Run by submitting the tool output from the function(s) you call. 
        ### Pass the tool_call_id referenced in the required_action object to match output to each function call.
        tool_outputs.append(
          { 
            "tool_call_id": tool_call.id,
            "output": str(function_response)
          }
        )
        print("Tool output: ", tool_outputs)
        run = client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs = tool_outputs
        )

    elif run.status == "failed":
        print(run.last_error)
        
    else:
        pass

In [9]:
user_question ="""What countries can I visit?"""
run_assistant(user_question)

requires_action
Function Name:  get_countries
Function Arguments:  {}
Function Response:  
        United States
        Australia
        France
    
Tool output:  [{'tool_call_id': 'call_ciwREOkfNVBhrvTyKLR9vzgx', 'output': '\n        United States\n        Australia\n        France\n    '}]
completed
You can visit the following countries:

1. United States
2. Australia
3. France


## Step 7: Append Messages

In [10]:
user_question ="""What is the weather like in the United States?"""
run_assistant(user_question)

requires_action
Function Name:  get_weather
Function Arguments:  {'country': 'United States'}
Function Response:  
            The United States has a diverse climate. In general:
            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
            - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
        
Tool output:  [{'tool_call_id': 'call_o50LxwhgtTm2xvS3l4g5tin3', 'output': '\n            The United States has a diverse climate. In general:\n            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.\n            - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).\n        '}]
in_progress
completed
The weather in the United States varies greatly due

In [11]:
user_question ="""Will $500 fit for a 4 day vacation there?"""
run_assistant(user_question)

requires_action
Function Name:  get_travel_budget
Function Arguments:  {}
Function Response:  
        If country is not defined:
            Look at all the budgets per country
        United States Budget:
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        Australia Budget:
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes 

## Delete Assistant

In [12]:
response = client.beta.assistants.delete(assistant.id)